In [1]:
%matplotlib inline

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))



Sequence Models and Long-Short Term Memory Networks
===================================================

At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding *hidden state* $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.


LSTM's in Pytorch
~~~~~~~~~~~~~~~~~

Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [2]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#for own training data
import pickle
import numpy as np
import util

torch.manual_seed(1)

In [4]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [autograd.Variable(torch.randn((1, 3)))
          for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn((1, 1, 3))))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    #print(out)
    #print(hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time

# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(
    torch.randn((1, 1, 3))))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

Variable containing:
(0 ,.,.) = 
  0.2490 -0.0525  0.3253

(1 ,.,.) = 
  0.1655 -0.0304  0.3348

(2 ,.,.) = 
 -0.1104 -0.1085  0.7568

(3 ,.,.) = 
 -0.0148 -0.0855  0.4162

(4 ,.,.) = 
  0.0703 -0.1089  0.2071
[torch.FloatTensor of size 5x1x3]

(Variable containing:
(0 ,.,.) = 
  0.0703 -0.1089  0.2071
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  0.2099 -0.3541  0.9947
[torch.FloatTensor of size 1x1x3]
)


Example: An LSTM for Part-of-Speech Tagging
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.


Prepare data:



In [7]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"]),
    ("The player runs swiftly".split(), ["DET", "NN", "V", "RB"])
]
word_to_ix = {}
char_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            for c in word:
                if c not in char_to_ix:
                    char_to_ix[c] = len(char_to_ix)
print(word_to_ix)
print(char_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2, "RB":3}


# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8, 'player': 9, 'runs': 10, 'swiftly': 11}
{'T': 0, 'h': 1, 'e': 2, 'd': 3, 'o': 4, 'g': 5, 'a': 6, 't': 7, 'p': 8, 'l': 9, 'E': 10, 'v': 11, 'r': 12, 'y': 13, 'b': 14, 'k': 15, 'u': 16, 'n': 17, 's': 18, 'w': 19, 'i': 20, 'f': 21}


Create the model:



In [6]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [102]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
inputs = prepare_sequence(training_data[2][0], word_to_ix)
tag_scores = model(inputs)
# The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#  for word i. The predicted tag is the maximum scoring tag.
# Here, we can see the predicted sequence below is 0 1 2 0 1
# since 0 is index of the maximum value of row 1,
# 1 is the index of maximum value of row 2, etc.
# Which is DET NOUN VERB DET NOUN, the correct sequence!
print(tag_scores)

Variable containing:
-1.2488 -1.1109 -1.4853 -1.8487
-1.3470 -1.1381 -1.4420 -1.6976
-1.3462 -1.1293 -1.4702 -1.6785
-1.1764 -1.1414 -1.5794 -1.7949
-1.0546 -1.1462 -1.6832 -1.9101
[torch.FloatTensor of size 5x4]

Variable containing:
-0.3086 -1.8983 -2.7450 -2.9676
-3.1102 -0.4139 -1.7693 -2.0884
-4.4105 -2.2488 -0.4643 -1.3714
-3.4807 -1.7435 -1.0211 -0.8345
[torch.FloatTensor of size 4x4]



Exercise: Augmenting the LSTM part-of-speech tagger with character-level features
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.




In [8]:
def prepare_sequence_char(seq, to_ix, char_to_ix):
    idxs = [to_ix[w] for w in seq]
    word_tensor = torch.LongTensor(idxs)
    return autograd.Variable(word_tensor), [autograd.Variable(torch.LongTensor([char_to_ix[c] for c in w])) for w in seq]

WORD_EMBEDDING_DIM = 64
CHAR_EMBEDDING_DIM = 10
HIDDEN_DIM = 6
CHAR_HIDDEN_DIM = 6

In [9]:
#%%writefile train.py
class LSTMTagger_Enhanced(nn.Module):
    def __init__(self, word_embedding_dim, char_embedding_dim, hidden_dim, char_hidden_dim, vocab_size, char_size, tagset_size):
        super(LSTMTagger_Enhanced, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        self.char_embedding = nn.Embedding(char_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim, char_hidden_dim)
        self.char_hidden = self.init_hidden(char_hidden_dim)
        self.lstm = nn.LSTM(char_hidden_dim+word_embedding_dim, hidden_dim)
        self.hidden = self.init_hidden(hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
    def init_hidden(self, char_hidden_dim):
        return (autograd.Variable(torch.zeros(1, 1, char_hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, char_hidden_dim)))
    
    def forward(self, sentence, charsets):
        charset_lstm_out = []
        for charset in charsets:
            char_embeds = self.char_embedding(charset)
            char_lstm_out, char_hidden = self.char_lstm(
                char_embeds.view(len(charset), 1, -1), self.char_hidden)
            #take the last hidden 
            charset_lstm_out.append(char_lstm_out[-1].view(1, -1))
        charset_lstm_out = torch.cat(charset_lstm_out)
        word_embeds = self.word_embedding(sentence)
        embeds = torch.cat([charset_lstm_out, word_embeds], dim=1)
        lstm_out, hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [9]:
with open('data/2012_training_data_py3.pickle', 'rb') as f:
    training_data = pickle.load(f)
word_to_ix = {}
char_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            for c in word:
                if c not in char_to_ix:
                    char_to_ix[c] = len(char_to_ix)
word_to_ix['UNKONWN'] = len(word_to_ix)
char_to_ix['UNKONWN'] = len(char_to_ix)
print(word_to_ix)
print(char_to_ix)
tag_to_ix = { 'O':0,

           'B-problem':1, 'B-test':2, 'B-treatment':3, 'B-occurrence':4, 'B-clinical_dept':5, 'B-evidential':6,

           'I-problem':7, 'I-test':8, 'I-treatment':9, 'I-occurrence':10, 'I-clinical_dept':11, 'I-evidential':12,

         }

{'Admission': 0, 'Date': 1, ':': 2, '09/29/1993': 3, 'Discharge': 4, '10/04/1993': 5, 'HISTORY': 6, 'OF': 7, 'PRESENT': 8, 'ILLNESS': 9, 'The': 10, 'patient': 11, 'is': 12, 'a': 13, '28-year-old': 14, 'woman': 15, 'who': 16, 'HIV': 17, 'positive': 18, 'for': 19, 'two': 20, 'years': 21, '.': 22, 'She': 23, 'presented': 24, 'with': 25, 'left': 26, 'upper': 27, 'quadrant': 28, 'pain': 29, 'as': 30, 'well': 31, 'nausea': 32, 'and': 33, 'vomiting': 34, 'which': 35, 'long-standing': 36, 'complaint': 37, 'was': 38, 'diagnosed': 39, 'in': 40, '1991': 41, 'during': 42, 'the': 43, 'birth': 44, 'of': 45, 'her': 46, 'child': 47, 'claims': 48, 'she': 49, 'does': 50, 'not': 51, 'know': 52, 'why': 53, 'from': 54, 'Maryland': 55, ',': 56, 'apparently': 57, 'had': 58, 'no': 59, 'blood': 60, 'transfusions': 61, 'before': 62, 'children': 63, 'so': 64, 'it': 65, 'presumed': 66, 'heterosexual': 67, 'transmission': 68, 'At': 69, 'that': 70, 'time': 71, 'also': 72, 'cat': 73, 'scratch': 74, 'fever': 75, 'res

In [4]:

model = LSTMTagger_Enhanced(WORD_EMBEDDING_DIM, CHAR_EMBEDDING_DIM, HIDDEN_DIM, CHAR_HIDDEN_DIM, len(word_to_ix), len(char_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

inputs, inputs_char = prepare_sequence_char(training_data[0][0], word_to_ix, char_to_ix)
tag_scores = model(inputs, inputs_char)
print(tag_scores)
hist_loss = []
for epoch in range(10):
    i = 0
    for sentence, tags in training_data:
        model.zero_grad()
        model.char_hidden = model.init_hidden(CHAR_HIDDEN_DIM)
        model.hidden = model.init_hidden(HIDDEN_DIM)
        if sentence == []:
            continue
        word_t, char_t = prepare_sequence_char(sentence, word_to_ix, char_to_ix)
        
        tag_scores = model.forward(word_t, char_t)
        targets = prepare_sequence(tags, tag_to_ix)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        #if i % 1000 == 0:
        print(i, loss)
        i += 1
    print(loss.data.numpy())
    hist_loss.append(loss.data.numpy()) 
    
inputs, inputs_char = prepare_sequence_char(training_data[2][0], word_to_ix, char_to_ix)
tag_scores = model(inputs, inputs_char)
print(tag_scores)

NameError: name 'LSTMTagger_Enhanced' is not defined

In [16]:
l = [[1,2,1], [2,3,4]]
t = torch.LongTensor(l)
t.numpy()
#v = autograd.Variable(t)
#for vv in v:
#    print(vv)

array([[1, 2, 1],
       [2, 3, 4]])

In [37]:
for i in range(1):
    test_sentences, test_labels = util.file_to_list('train/11.txt', 'train/11.con')
    for j in range(len(test_sentences)):
        sentence = test_sentences[j]
        label = test_labels[j]
        print(sentence)
        #print(label)
        #print(tags_to_i2b2(j, sentence, label))
        origin = util.tags_to_i2b2(j, sentence, label)
        if sentence == []:
            continue
        inputs, inputs_char = prepare_sequence_char(sentence, word_to_ix, char_to_ix)
        tag_scores = model(inputs, inputs_char)
        #print(scores_to_tags(tag_scores, tag_to_ix))
        print(util.tags_to_i2b2(j, sentence, scores_to_tags(tag_scores, tag_to_ix)))
        pred = util.tags_to_i2b2(j, sentence, util.scores_to_tags(tag_scores, tag_to_ix))
        print(set(origin) & set(pred))
        print("\n")

train/11.txt
['Admission', 'Date', ':']


AttributeError: module 'util' has no attribute 'tags_to_i2b2'

In [63]:
#torch.save(model, 'test.model')
# import glob
# from predict import *
# train = glob.glob('train/*.txt')
# con = glob.glob('train/*.con')
# train = sorted(train)
# con = sorted(con)
# predict('epoch350_dim100.model', train[1], con[1], '.')
# %run predict.py train/11.txt train/11.con lstm/epoch200.model
%run predict_lstm.py test_questions.txt test_questions_ground_truth.con lstm/epoch350.model
#torch.load('epoch1.model')
#%run train.py epoch401.model
#predict(model, 'train/11.txt', 'train/11.con', '.')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrie

0
['How', 'many', 'patients', 'suffer', 'from', 'Diabetes']
['c="Diabetes" 0:5 0:5||t="problem"']
['c="Diabetes" 1:5 1:5||t="problem"']
{'c="Diabetes" 1:5 1:5||t="problem"'}


1
['How', 'many', 'patients', 'suffer', 'from', 'heart', 'failure']
['c="heart failure" 1:5 1:6||t="problem"']
['c="heart failure" 2:5 2:6||t="test"']
set()


2
['How', 'many', 'patients', 'have', 'Diabetes']
['c="Diabetes" 2:4 2:4||t="problem"']
['c="Diabetes" 3:4 3:4||t="problem"']
{'c="Diabetes" 3:4 3:4||t="problem"'}


3
['How', 'many', 'patients', 'have', 'heart', 'failure']
['c="heart failure" 3:4 3:5||t="problem"']
['c="heart failure" 4:4 4:5||t="problem"']
{'c="heart failure" 4:4 4:5||t="problem"'}


4
['How', 'many', 'Diabetes', 'patients', 'have', 'been', 'admitted']
['c="Diabetes" 4:2 4:2||t="problem"']
[]
set()


5
['How', 'many', 'heart', 'failure', 'patients', 'have', 'been', 'admitted']
['c="heart failure" 5:2 5:3||t="problem"']
[]
set()


6
['How', 'many', 'patients', 'have', 'been', 'administered

['c="expired" 62:3 62:3||t="treatment"', 'c="heart failure" 62:6 62:7||t="problem"']
{'c="heart failure" 62:6 62:7||t="problem"'}


62
['Which', 'patients', 'expired', 'due', 'to', 'Diabetes', 'in', 'the', 'hospital']
['c="Diabetes" 62:5 62:5||t="problem"']
['c="expired" 63:2 63:2||t="problem"', 'c="Diabetes in the hospital" 63:5 63:8||t="problem"']
set()


63
['Which', 'patients', 'expired', 'due', 'to', 'heart', 'failure', 'in', 'the', 'hospital']
['c="heart failure" 63:5 63:6||t="problem"']
['c="expired" 64:2 64:2||t="problem"', 'c="heart failure" 64:5 64:6||t="test"']
set()


64
['Which', 'patients', 'expired', 'because', 'of', 'Diabetes', 'in', 'the', 'hospital']
['c="Diabetes" 64:5 64:5||t="problem"']
['c="expired" 65:2 65:2||t="problem"', 'c="Diabetes" 65:5 65:5||t="problem"']
{'c="Diabetes" 65:5 65:5||t="problem"'}


65
['Which', 'patients', 'expired', 'because', 'of', 'heart', 'failure', 'in', 'the', 'hospital']
['c="heart failure" 65:5 65:6||t="problem"']
['c="expired" 66:2 6

['c="heart failure" 117:4 117:5||t="problem"']
{'c="heart failure" 117:4 117:5||t="problem"'}


117
['How', 'many', 'patients', 'with', 'heart', 'failure', 'under', '3.54', 'age']
['c="heart failure" 117:4 117:5||t="problem"']
['c="heart failure" 118:4 118:5||t="problem"']
{'c="heart failure" 118:4 118:5||t="problem"'}


118
['How', 'many', 'Diabetes', 'patients', 'were', 'under', '1.5', 'age']
['c="Diabetes" 118:2 118:2||t="problem"']
[]
set()


119
['How', 'many', 'Diabetes', 'patients', 'were', 'under', '4.9', 'age']
['c="Diabetes" 119:2 119:2||t="problem"']
[]
set()


120
['How', 'many', 'Diabetes', 'patients', 'were', 'under', '3.54', 'age']
['c="Diabetes" 120:2 120:2||t="problem"']
[]
set()


121
['How', 'many', 'heart', 'failure', 'patients', 'were', 'under', '1.5', 'age']
['c="heart failure" 121:2 121:3||t="problem"']
[]
set()


122
['How', 'many', 'heart', 'failure', 'patients', 'were', 'under', '4.9', 'age']
['c="heart failure" 122:2 122:3||t="problem"']
[]
set()


123
['How'

['c="Diabetes" 173:5 173:5||t="problem"']
{'c="Diabetes" 173:5 173:5||t="problem"'}


173
['How', 'many', 'patients', 'suffering', 'from', 'heart', 'failure', 'were', 'between', '10', 'and', '49', 'years']
['c="heart failure" 173:5 173:6||t="problem"']
['c="heart failure" 174:5 174:6||t="test"']
set()


174
['How', 'many', 'patients', 'suffering', 'from', 'Diabetes', 'were', 'between', '30', 'and', '79', 'years']
['c="Diabetes" 174:5 174:5||t="problem"']
['c="Diabetes" 175:5 175:5||t="problem"']
{'c="Diabetes" 175:5 175:5||t="problem"'}


175
['How', 'many', 'patients', 'suffering', 'from', 'heart', 'failure', 'were', 'between', '30', 'and', '79', 'years']
['c="heart failure" 175:5 175:6||t="problem"']
['c="heart failure" 176:5 176:6||t="test"']
set()


176
['How', 'many', 'patients', 'with', 'Diabetes', 'were', 'between', '10', 'and', '49', 'years']
['c="Diabetes" 176:4 176:4||t="problem"']
['c="Diabetes" 177:4 177:4||t="problem"']
{'c="Diabetes" 177:4 177:4||t="problem"'}


177
['How

['c="positive labs" 225:4 225:5||t="problem"', 'c="radiograph" 225:7 225:7||t="problem"']
set()


225
['How', 'many', 'patients', 'had', 'positive', 'results', 'for', 'radiograph']
['c="radiograph" 225:7 225:7||t="test"']
['c="positive results" 226:4 226:5||t="problem"', 'c="radiograph" 226:7 226:7||t="problem"']
set()


226
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'performed', 'and', 'had', 'positive', 'results']
['c="radiograph" 226:4 226:4||t="test"']
['c="positive" 227:9 227:9||t="problem"']
set()


227
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'and', 'had', 'positive', 'results']
['c="radiograph" 227:6 227:6||t="test"']
['c="positive" 228:9 228:9||t="problem"']
set()


228
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'negative']
['c="radiograph" 228:6 228:6||t="test"']
[]
set()


229
['How', 'many', 'patients', 'had', 'negative', 'radiograph', 'test']
['c="radiograph" 229:5 229:5||t="test"']
['c="negative radiograph te

[]
set()


289
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'whose', 'results', 'were', 'below', '4.9']
['c="radiograph" 289:6 289:6||t="test"']
[]
set()


290
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'whose', 'results', 'were', 'below', '3.54']
['c="radiograph" 290:6 290:6||t="test"']
[]
set()


291
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'below', '1.5']
['c="radiograph" 291:4 291:4||t="test"']
[]
set()


292
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'below', '4.9']
['c="radiograph" 292:4 292:4||t="test"']
[]
set()


293
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'below', '3.54']
['c="radiograph" 293:4 293:4||t="test"']
[]
set()


294
['How', 'many', 'patients', 'had', 'labs', 'for', 'radiograph', 'whose', 'results', 'were', 'below', '1.5']
['c="radiograph" 294:6 294:6||t="test"']
['c="labs" 295:4 295:4||t="te

['c="immunodeficiency" 351:4 351:4||t="treatment"']
set()


351
['How', 'many', 'people', 'with', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'with', 'results', 'greater', 'than', '3.54']
['c="immunodeficiency" 351:4 351:4||t="problem"', 'c="HIV" 351:8 351:8||t="test"']
['c="immunodeficiency" 352:4 352:4||t="treatment"']
set()


352
['How', 'many', 'people', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'values', 'greater', 'than', '1.5']
['c="immunodeficiency" 352:5 352:5||t="problem"', 'c="HIV" 352:9 352:9||t="test"']
['c="HIV" 353:9 353:9||t="problem"']
set()


353
['How', 'many', 'people', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'values', 'greater', 'than', '4.9']
['c="immunodeficiency" 353:5 353:5||t="problem"', 'c="HIV" 353:9 353:9||t="test"']
['c="HIV" 354:9 354:9||t="problem"']
set()


354
['How', 'many', 'people', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'values', 'greater', 'th

['c="Digoxin dosage above" 394:5 394:7||t="treatment"']
set()


394
['How', 'many', 'people', 'were', 'treated', 'with', 'Januvia', 'dosage', 'lesser', 'than', '1.5']
['c="Januvia" 394:6 394:6||t="treatment"']
['c="Januvia dosage lesser" 395:6 395:8||t="treatment"']
set()


395
['How', 'many', 'people', 'were', 'treated', 'with', 'Januvia', 'dosage', 'lesser', 'than', '4.9']
['c="Januvia" 395:6 395:6||t="treatment"']
['c="Januvia dosage lesser" 396:6 396:8||t="treatment"']
set()


396
['How', 'many', 'people', 'were', 'treated', 'with', 'Januvia', 'dosage', 'lesser', 'than', '3.54']
['c="Januvia" 396:6 396:6||t="treatment"']
['c="Januvia dosage lesser" 397:6 397:8||t="treatment"']
set()


397
['How', 'many', 'people', 'were', 'treated', 'with', 'Digoxin', 'dosage', 'lesser', 'than', '1.5']
['c="Digoxin" 397:6 397:6||t="treatment"']
['c="Digoxin dosage lesser than 1.5" 398:6 398:10||t="treatment"']
set()


398
['How', 'many', 'people', 'were', 'treated', 'with', 'Digoxin', 'dosage', 'le

In [3]:
%run ner_to_seq.py test_questions.txt test_questions_cliner.con

How many patients suffer from Diabetes
{}

How many patients suffer from PROBLEM_1
{'PROBLEM_1': 'heart failure'}

How many patients have Diabetes
{}

How many patients have PROBLEM_1
{'PROBLEM_1': 'heart failure'}

How many Diabetes patients have been OCCURRENCE_1
{'OCCURRENCE_1': 'admitted'}

How PROBLEM_1 patients have been OCCURRENCE_1
{'PROBLEM_1': 'many heart failure', 'OCCURRENCE_1': 'admitted'}

How many patients have been administered Januvia
{}

How many patients have been TREATMENT_1
{'TREATMENT_1': 'administered Digoxin'}

How many patients have been given Januvia
{}

How many patients have been given TREATMENT_1
{'TREATMENT_1': 'Digoxin'}

What problems did patients given PROBLEM_1 from
{'PROBLEM_1': 'Januvia suffer'}

What problems did patients given TREATMENT_1 from
{'TREATMENT_1': 'Digoxin suffer'}

Which problems are treated with PROBLEM_1
{'PROBLEM_1': 'Januvia'}

Which problems are treated with TREATMENT_1
{'TREATMENT_1': 'Digoxin'}

Which drugs are used to TREATMENT

In [23]:
%run ner_to_seq.py test_questions.txt test_questions.con
#util.file_to_list('test_questions.txt','test_questions_cliner.con')

How many patients suffer from Diabetes
{}

How many patients suffer from heart failure
{}

How many patients have Diabetes
{}

How many patients have heart failure
{}

How many Diabetes patients have been admitted
{}

How many heart failure patients have been admitted
{}

How many patients have been administered Januvia
{}

How many patients have been administered Digoxin
{}

How many patients have been given Januvia
{}

How many patients have been given Digoxin
{}

What problems did patients given OCCURRENCE_1 suffer from
{'OCCURRENCE_1': 'Januvia'}

What problems did patients given TREATMENT_1 from
{'TREATMENT_1': 'Digoxin suffer'}

Which PROBLEM_1 are treated with Januvia
{'PROBLEM_1': 'problems'}

Which PROBLEM_1 are treated with Digoxin
{'PROBLEM_1': 'problems'}

Which PROBLEM_1 are used to treat Diabetes
{'PROBLEM_1': 'drugs'}

Which PROBLEM_1 are used to treat heart failure
{'PROBLEM_1': 'drugs'}

Which PROBLEM_1 are administered to treat Diabetes
{'PROBLEM_1': 'drugs'}

Which P

In [12]:
import glob
train = glob.glob('train/*.con')
pred = glob.glob('lstm_pred/*.con')
num_pred = 0
num_total = 0
num_inter = 0
for i in range(len(train)):
    n_t, n_p, n_i = util.evaluate(train[i], pred[i])
    num_pred += n_p
    num_total += n_t
    num_inter += n_i

precision:0.54
recall:0.47
f measure:0.50:
precision:0.80
recall:0.66
f measure:0.72:
precision:0.64
recall:0.58
f measure:0.61:
precision:0.69
recall:0.65
f measure:0.67:
precision:0.66
recall:0.58
f measure:0.62:
precision:0.79
recall:0.69
f measure:0.74:
precision:0.82
recall:0.72
f measure:0.77:
precision:0.73
recall:0.64
f measure:0.68:
precision:0.86
recall:0.73
f measure:0.79:
precision:0.57
recall:0.57
f measure:0.57:
precision:0.80
recall:0.80
f measure:0.80:
precision:0.74
recall:0.58
f measure:0.65:
precision:0.62
recall:0.59
f measure:0.60:
precision:0.70
recall:0.67
f measure:0.69:
precision:0.68
recall:0.51
f measure:0.58:
precision:0.60
recall:0.51
f measure:0.55:
precision:0.65
recall:0.63
f measure:0.64:
precision:0.69
recall:0.58
f measure:0.63:
precision:0.68
recall:0.58
f measure:0.62:
precision:0.68
recall:0.63
f measure:0.66:
precision:0.76
recall:0.67
f measure:0.72:
precision:0.73
recall:0.64
f measure:0.68:
precision:0.84
recall:0.73
f measure:0.78:
precision:0

In [16]:
precision = num_inter / num_pred
recall = num_inter / num_total
print("precision:{0:.2f}".format(precision))
print("recall:{0:.2f}".format(recall))
print("f measure:{0:.2f}".format(2 * precision * recall / (precision + recall)))

precision:0.70
recall:0.64
f measure:0.67


In [17]:
import glob
train = glob.glob('train/*.con')
pred = glob.glob('pred/*.con')
num_pred = 0
num_total = 0
num_inter = 0
for i in range(len(train)):
    n_t, n_p, n_i = util.evaluate(train[i], pred[i])
    num_pred += n_p
    num_total += n_t
    num_inter += n_i

precision:0.72
recall:0.75
f measure:0.73:
precision:0.75
recall:0.77
f measure:0.76:
precision:0.77
recall:0.75
f measure:0.76:
precision:0.65
recall:0.66
f measure:0.66:
precision:0.63
recall:0.62
f measure:0.63:
precision:0.83
recall:0.82
f measure:0.83:
precision:0.57
recall:0.57
f measure:0.57:
precision:0.79
recall:0.81
f measure:0.80:
precision:0.68
recall:0.68
f measure:0.68:
precision:0.70
recall:0.70
f measure:0.70:
precision:0.76
recall:0.76
f measure:0.76:
precision:0.69
recall:0.68
f measure:0.69:
precision:0.70
recall:0.67
f measure:0.68:
precision:0.80
recall:0.80
f measure:0.80:
precision:0.72
recall:0.71
f measure:0.72:
precision:0.83
recall:0.82
f measure:0.82:
precision:0.53
recall:0.49
f measure:0.50:
precision:0.75
recall:0.70
f measure:0.73:
precision:0.73
recall:0.72
f measure:0.73:
precision:0.49
recall:0.51
f measure:0.50:
precision:0.64
recall:0.64
f measure:0.64:
precision:0.51
recall:0.51
f measure:0.51:
precision:0.65
recall:0.67
f measure:0.66:
precision:0

ZeroDivisionError: float division by zero

In [18]:
precision = num_inter / num_pred
recall = num_inter / num_total
print("precision:{0:.2f}".format(precision))
print("recall:{0:.2f}".format(recall))
print("f measure:{0:.2f}".format(2 * precision * recall / (precision + recall)))

precision:0.69
recall:0.69
f measure:0.69


In [6]:
util.evaluate('test_questions_ground_truth.con', 'test_questions.con')

precision:0.32
recall:0.20
f measure:0.25:


(466, 293, 94)

# BiLSTM and softmax

In [64]:
%run predict_bilstm.py train/1.txt train/1.con bilstm_200_50_20_15/epoch50.model

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class '__main__.BiLSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original

0
['Admission', 'Date', ':']
['c="Admission" 0:0 0:0||t="occurrence"']
[]
set()


1
['09/29/1993']
[]
[]
set()


2
['Discharge', 'Date', ':']
['c="Discharge" 2:0 2:0||t="occurrence"']
[]
set()


3
['10/04/1993']
[]
[]
set()


4
['HISTORY', 'OF', 'PRESENT', 'ILLNESS', ':']
[]
[]
set()


5
['The', 'patient', 'is', 'a', '28-year-old', 'woman', 'who', 'is', 'HIV', 'positive', 'for', 'two', 'years', '.']
['c="HIV positive" 5:8 5:9||t="problem"']
[]
set()


6
['She', 'presented', 'with', 'left', 'upper', 'quadrant', 'pain', 'as', 'well', 'as', 'nausea', 'and', 'vomiting', 'which', 'is', 'a', 'long-standing', 'complaint', '.']
['c="presented" 6:1 6:1||t="evidential"', 'c="left upper quadrant pain" 6:3 6:6||t="problem"', 'c="nausea" 6:10 6:10||t="problem"', 'c="vomiting" 6:12 6:12||t="problem"', 'c="a long-standing complaint" 6:15 6:17||t="problem"']
['c="left upper quadrant pain" 7:3 7:6||t="problem"', 'c="nausea" 7:10 7:10||t="problem"', 'c="vomiting" 7:12 7:12||t="problem"', 'c="a long-stan

In [65]:
%run predict_bilstm.py test_questions.txt test_questions_ground_truth.con bilstm_200_50_20_15/epoch50.model

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class '__main__.BiLSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original

0
['How', 'many', 'patients', 'suffer', 'from', 'Diabetes']
['c="Diabetes" 0:5 0:5||t="problem"']
['c="patients suffer" 1:2 1:3||t="problem"', 'c="Diabetes" 1:5 1:5||t="problem"']
{'c="Diabetes" 1:5 1:5||t="problem"'}


1
['How', 'many', 'patients', 'suffer', 'from', 'heart', 'failure']
['c="heart failure" 1:5 1:6||t="problem"']
['c="patients suffer" 2:2 2:3||t="problem"']
set()


2
['How', 'many', 'patients', 'have', 'Diabetes']
['c="Diabetes" 2:4 2:4||t="problem"']
['c="many" 3:1 3:1||t="test"', 'c="Diabetes" 3:4 3:4||t="problem"']
{'c="Diabetes" 3:4 3:4||t="problem"'}


3
['How', 'many', 'patients', 'have', 'heart', 'failure']
['c="heart failure" 3:4 3:5||t="problem"']
['c="How" 4:0 4:0||t="test"']
set()


4
['How', 'many', 'Diabetes', 'patients', 'have', 'been', 'admitted']
['c="Diabetes" 4:2 4:2||t="problem"']
['c="Diabetes" 5:2 5:2||t="test"']
set()


5
['How', 'many', 'heart', 'failure', 'patients', 'have', 'been', 'admitted']
['c="heart failure" 5:2 5:3||t="problem"']
['c="How"

['c="heart failure" 56:5 56:6||t="problem"']
{'c="heart failure" 56:5 56:6||t="problem"'}


56
['Which', 'patients', 'died', 'with', 'Diabetes', 'in', 'the', 'hospital']
['c="Diabetes" 56:4 56:4||t="problem"']
['c="Diabetes" 57:4 57:4||t="test"']
set()


57
['Which', 'patients', 'died', 'with', 'heart', 'failure', 'in', 'the', 'hospital']
['c="heart failure" 57:4 57:5||t="problem"']
['c="heart failure" 58:4 58:5||t="problem"']
{'c="heart failure" 58:4 58:5||t="problem"'}


58
['How', 'many', 'patients', 'expired', 'due', 'to', 'Diabetes', 'in', 'the', 'hospital']
['c="Diabetes" 58:6 58:6||t="problem"']
['c="many" 59:1 59:1||t="test"', 'c="Diabetes" 59:6 59:6||t="problem"']
{'c="Diabetes" 59:6 59:6||t="problem"'}


59
['How', 'many', 'patients', 'expired', 'due', 'to', 'heart', 'failure', 'in', 'the', 'hospital']
['c="heart failure" 59:6 59:7||t="problem"']
['c="expired" 60:3 60:3||t="problem"']
set()


60
['How', 'many', 'patients', 'expired', 'because', 'of', 'Diabetes', 'in', 'the', 

['c="patients" 104:10 104:10||t="problem"', 'c="Medicaid" 104:12 104:12||t="problem"']
set()


104
['What', 'was', 'the', 'average', 'length', 'of', 'stay', 'in', 'ICU', 'for', 'patients', 'with', 'no', 'coverage']
[]
['c="patients" 105:10 105:10||t="problem"']
set()


105
['What', 'was', 'the', 'max', 'length', 'of', 'stay', 'in', 'ICU', 'for', 'patients', 'with', 'no', 'coverage']
[]
['c="patients" 106:10 106:10||t="problem"']
set()


106
['What', 'was', 'the', 'average', 'length', 'of', 'stay', 'in', 'ICU', 'for', 'patients', 'with', 'no', 'coverage']
[]
['c="patients" 107:10 107:10||t="problem"']
set()


107
['How', 'many', 'BLACK', 'patients', 'suffer', 'from', 'immunodeficiency']
['c="immunodeficiency" 107:6 107:6||t="problem"']
['c="patients suffer" 108:3 108:4||t="problem"', 'c="immunodeficiency" 108:6 108:6||t="treatment"']
set()


108
['How', 'many', 'patients', 'have', 'immunodeficiency', 'who', 'are', 'BLACK']
['c="immunodeficiency" 108:4 108:4||t="problem"']
['c="many" 109

['c="many patients" 153:1 153:2||t="problem"', 'c="heart failure" 153:4 153:5||t="problem"']
{'c="heart failure" 153:4 153:5||t="problem"'}


153
['How', 'many', 'patients', 'with', 'heart', 'failure', 'over', '3.54', 'years']
['c="heart failure" 153:4 153:5||t="problem"']
['c="many patients" 154:1 154:2||t="problem"']
set()


154
['How', 'many', 'Diabetes', 'patients', 'were', 'over', '1.5', 'years']
['c="Diabetes" 154:2 154:2||t="problem"']
['c="Diabetes" 155:2 155:2||t="test"']
set()


155
['How', 'many', 'Diabetes', 'patients', 'were', 'over', '4.9', 'years']
['c="Diabetes" 155:2 155:2||t="problem"']
['c="Diabetes" 156:2 156:2||t="test"']
set()


156
['How', 'many', 'Diabetes', 'patients', 'were', 'over', '3.54', 'years']
['c="Diabetes" 156:2 156:2||t="problem"']
['c="Diabetes" 157:2 157:2||t="test"']
set()


157
['How', 'many', 'heart', 'failure', 'patients', 'were', 'over', '1.5', 'years']
['c="heart failure" 157:2 157:3||t="problem"']
['c="How" 158:0 158:0||t="test"']
set()


15

...

['c="many" 280:1 280:1||t="test"', 'c="radiograph" 280:6 280:6||t="problem"']
set()


280
['How', 'many', 'patients', 'had', 'results', 'for', 'radiograph', 'lesser', 'than', '4.9']
['c="radiograph" 280:6 280:6||t="test"']
['c="many" 281:1 281:1||t="test"', 'c="radiograph" 281:6 281:6||t="problem"']
set()


281
['How', 'many', 'patients', 'had', 'results', 'for', 'radiograph', 'lesser', 'than', '3.54']
['c="radiograph" 281:6 281:6||t="test"']
['c="many" 282:1 282:1||t="test"']
set()


282
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'performed', 'and', 'whose', 'results', 'were', 'lesser', 'than', '1.5']
['c="radiograph" 282:4 282:4||t="test"']
['c="many" 283:1 283:1||t="test"', 'c="radiograph test" 283:4 283:5||t="test"', 'c="whose" 283:8 283:8||t="problem"']
set()


283
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'performed', 'and', 'whose', 'results', 'were', 'lesser', 'than', '4.9']
['c="radiograph" 283:4 283:4||t="test"']
['c="many" 284:1 284:1||t="test"',

['c="many patients" 317:1 317:2||t="test"']
set()


317
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'and', 'had', 'results', 'between', '30', 'and', '79']
['c="radiograph" 317:6 317:6||t="test"']
['c="many patients" 318:1 318:2||t="test"']
set()


318
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'whose', 'results', 'were', 'between', '10', 'and', '49']
['c="radiograph" 318:6 318:6||t="test"']
['c="many patients" 319:1 319:2||t="test"', 'c="radiograph whose" 319:6 319:7||t="treatment"']
set()


319
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'whose', 'results', 'were', 'between', '30', 'and', '79']
['c="radiograph" 319:6 319:6||t="test"']
['c="many patients" 320:1 320:2||t="test"', 'c="radiograph whose" 320:6 320:7||t="treatment"']
set()


320
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'between', '10', 'and', '49']
['c="radiograph" 320:4 320:4||t="test"']
['c="many" 321:1 321:1|

['c="many" 353:1 353:1||t="problem"', 'c="immunodeficiency" 353:5 353:5||t="treatment"', 'c="HIV" 353:9 353:9||t="problem"']
set()


353
['How', 'many', 'people', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'values', 'greater', 'than', '4.9']
['c="immunodeficiency" 353:5 353:5||t="problem"', 'c="HIV" 353:9 353:9||t="test"']
['c="many" 354:1 354:1||t="problem"', 'c="immunodeficiency" 354:5 354:5||t="treatment"']
set()


354
['How', 'many', 'people', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'values', 'greater', 'than', '3.54']
['c="immunodeficiency" 354:5 354:5||t="problem"', 'c="HIV" 354:9 354:9||t="test"']
['c="many" 355:1 355:1||t="problem"', 'c="immunodeficiency" 355:5 355:5||t="treatment"']
set()


355
['How', 'many', 'patients', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'below', '1.5']
['c="immunodeficiency" 355:5 355:5||t="problem"', 'c="HIV" 355:9 355:9||t="test"']
['c="immunodeficiency" 3

['c="people" 392:2 392:2||t="problem"', 'c="Digoxin" 392:5 392:5||t="test"']
set()


392
['How', 'many', 'people', 'were', 'given', 'Digoxin', 'dosage', 'above', '4.9', 'in', 'the', 'hospital']
['c="Digoxin" 392:5 392:5||t="treatment"']
['c="people" 393:2 393:2||t="problem"', 'c="Digoxin" 393:5 393:5||t="test"']
set()


393
['How', 'many', 'people', 'were', 'given', 'Digoxin', 'dosage', 'above', '3.54', 'in', 'the', 'hospital']
['c="Digoxin" 393:5 393:5||t="treatment"']
['c="people" 394:2 394:2||t="problem"', 'c="Digoxin" 394:5 394:5||t="test"']
set()


394
['How', 'many', 'people', 'were', 'treated', 'with', 'Januvia', 'dosage', 'lesser', 'than', '1.5']
['c="Januvia" 394:6 394:6||t="treatment"']
[]
set()


395
['How', 'many', 'people', 'were', 'treated', 'with', 'Januvia', 'dosage', 'lesser', 'than', '4.9']
['c="Januvia" 395:6 395:6||t="treatment"']
['c="How" 396:0 396:0||t="test"']
set()


396
['How', 'many', 'people', 'were', 'treated', 'with', 'Januvia', 'dosage', 'lesser', 'than',

['c="people" 434:2 434:2||t="problem"', 'c="Digoxin" 434:5 434:5||t="test"']
set()


434
[]
[]
precision:0.14
recall:0.20
f measure:0.16:


# BiLSTM and crf

In [84]:
%run predict_crf.py test_input.txt test_input.txt lstm_crf/epoch100.model

0
['How', 'many', 'Diabetes', 'patients', 'are', 'younger', 'than', '3', 'years']
[]
['c="Diabetes" 1:2 1:2||t="problem"', 'c="younger" 1:5 1:5||t="problem"']
set()


1
['How', 'many', 'Diabetes', 'patients', 'are', 'there']
[]
['c="Diabetes" 2:2 2:2||t="problem"']
set()


2
['How', 'many', 'Diabetes', 'patients', 'died', 'last', 'year']
[]
['c="died" 3:4 3:4||t="occurrence"']
set()


3
['How', 'many', 'immunodeficiency', 'patients', 'have', 'been', 'tested', 'for', 'HIV']
[]
[]
set()


0 4 0


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class '__main__.BiLSTM_CRF' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the orig

ZeroDivisionError: division by zero

In [6]:
%run predict_crf_emb.py test_questions.txt test_questions_ground_truth.con lstm_crf/epoch100.model

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class '__main__.BiLSTM_CRF' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the orig

0
['How', 'many', 'patients', 'suffer', 'from', 'Diabetes']
['c="Diabetes" 1:5 1:5||t="problem"']
[]
set()


1
['How', 'many', 'patients', 'suffer', 'from', 'heart', 'failure']
['c="heart failure" 2:5 2:6||t="problem"']
['c="heart failure" 2:5 2:6||t="test"']
set()


2
['How', 'many', 'patients', 'have', 'Diabetes']
['c="Diabetes" 3:4 3:4||t="problem"']
['c="Diabetes" 3:4 3:4||t="problem"']
{'c="Diabetes" 3:4 3:4||t="problem"'}


3
['How', 'many', 'patients', 'have', 'heart', 'failure']
['c="heart failure" 4:4 4:5||t="problem"']
['c="heart failure" 4:4 4:5||t="treatment"']
set()


4
['How', 'many', 'Diabetes', 'patients', 'have', 'been', 'admitted']
['c="Diabetes" 5:2 5:2||t="problem"']
['c="Diabetes" 5:2 5:2||t="problem"']
{'c="Diabetes" 5:2 5:2||t="problem"'}


5
['How', 'many', 'heart', 'failure', 'patients', 'have', 'been', 'admitted']
['c="heart failure" 6:2 6:3||t="problem"']
['c="heart failure" 6:2 6:3||t="treatment"']
set()


6
['How', 'many', 'patients', 'have', 'been', 'admin

['c="Diabetes" 55:5 55:5||t="problem"']
{'c="Diabetes" 55:5 55:5||t="problem"'}


55
['How', 'many', 'patients', 'died', 'with', 'heart', 'failure', 'in', 'the', 'hospital']
['c="heart failure" 56:5 56:6||t="problem"']
['c="heart failure" 56:5 56:6||t="problem"']
{'c="heart failure" 56:5 56:6||t="problem"'}


56
['Which', 'patients', 'died', 'with', 'Diabetes', 'in', 'the', 'hospital']
['c="Diabetes" 57:4 57:4||t="problem"']
['c="Diabetes" 57:4 57:4||t="problem"', 'c="the hospital" 57:6 57:7||t="treatment"']
{'c="Diabetes" 57:4 57:4||t="problem"'}


57
['Which', 'patients', 'died', 'with', 'heart', 'failure', 'in', 'the', 'hospital']
['c="heart failure" 58:4 58:5||t="problem"']
['c="heart failure" 58:4 58:5||t="problem"']
{'c="heart failure" 58:4 58:5||t="problem"'}


58
['How', 'many', 'patients', 'expired', 'due', 'to', 'Diabetes', 'in', 'the', 'hospital']
['c="Diabetes" 59:6 59:6||t="problem"']
['c="Diabetes" 59:6 59:6||t="problem"']
{'c="Diabetes" 59:6 59:6||t="problem"'}


59
['Ho

['c="3.54" 115:6 115:6||t="problem"']
set()


115
['How', 'many', 'patients', 'with', 'heart', 'failure', 'under', '1.5', 'age']
['c="heart failure" 116:4 116:5||t="problem"']
['c="heart failure" 116:4 116:5||t="problem"']
{'c="heart failure" 116:4 116:5||t="problem"'}


116
['How', 'many', 'patients', 'with', 'heart', 'failure', 'under', '4.9', 'age']
['c="heart failure" 117:4 117:5||t="problem"']
['c="heart failure" 117:4 117:5||t="problem"']
{'c="heart failure" 117:4 117:5||t="problem"'}


117
['How', 'many', 'patients', 'with', 'heart', 'failure', 'under', '3.54', 'age']
['c="heart failure" 118:4 118:5||t="problem"']
['c="heart failure" 118:4 118:5||t="problem"']
{'c="heart failure" 118:4 118:5||t="problem"'}


118
['How', 'many', 'Diabetes', 'patients', 'were', 'under', '1.5', 'age']
['c="Diabetes" 119:2 119:2||t="problem"']
['c="Diabetes" 119:2 119:2||t="problem"']
{'c="Diabetes" 119:2 119:2||t="problem"'}


119
['How', 'many', 'Diabetes', 'patients', 'were', 'under', '4.9', 'age

['c="heart failure" 158:2 158:3||t="problem"']
{'c="heart failure" 158:2 158:3||t="problem"'}


158
['How', 'many', 'heart', 'failure', 'patients', 'were', 'over', '4.9', 'years']
['c="heart failure" 159:2 159:3||t="problem"']
['c="heart failure" 159:2 159:3||t="problem"']
{'c="heart failure" 159:2 159:3||t="problem"'}


159
['How', 'many', 'heart', 'failure', 'patients', 'were', 'over', '3.54', 'years']
['c="heart failure" 160:2 160:3||t="problem"']
['c="many heart failure" 160:1 160:3||t="problem"']
set()


160
['How', 'many', 'patients', 'suffering', 'from', 'Diabetes', 'were', 'over', '1.5', 'years']
['c="Diabetes" 161:5 161:5||t="problem"']
['c="Diabetes" 161:5 161:5||t="treatment"']
set()


161
['How', 'many', 'patients', 'suffering', 'from', 'Diabetes', 'were', 'over', '4.9', 'years']
['c="Diabetes" 162:5 162:5||t="problem"']
['c="Diabetes" 162:5 162:5||t="treatment"']
set()


162
['How', 'many', 'patients', 'suffering', 'from', 'Diabetes', 'were', 'over', '3.54', 'years']
['c="

['c="heart failure" 208:5 208:6||t="problem"']
{'c="heart failure" 208:5 208:6||t="problem"'}


208
['How', 'many', 'males', 'suffer', 'from', 'Diabetes']
['c="Diabetes" 209:5 209:5||t="problem"']
[]
set()


209
['How', 'many', 'males', 'suffer', 'from', 'heart', 'failure']
['c="heart failure" 210:5 210:6||t="problem"']
[]
set()


210
['How', 'many', 'male', 'patients', 'suffer', 'from', 'Diabetes']
['c="Diabetes" 211:6 211:6||t="problem"']
[]
set()


211
['How', 'many', 'male', 'patients', 'suffer', 'from', 'heart', 'failure']
['c="heart failure" 212:6 212:7||t="problem"']
['c="heart failure" 212:6 212:7||t="test"']
set()


212
['How', 'many', 'Diabetes', 'patients', 'are', 'males']
['c="Diabetes" 213:2 213:2||t="problem"']
['c="How" 213:0 213:0||t="test"', 'c="Diabetes" 213:2 213:2||t="problem"']
{'c="Diabetes" 213:2 213:2||t="problem"'}


213
['How', 'many', 'heart', 'failure', 'patients', 'are', 'males']
['c="heart failure" 214:2 214:3||t="problem"']
['c="failure" 214:3 214:3||t="p

['c="radiograph" 263:6 263:6||t="problem"']
set()


263
['How', 'many', 'patients', 'had', 'results', 'for', 'radiograph', 'above', '3.54']
['c="radiograph" 264:6 264:6||t="test"']
['c="radiograph above 3.54" 264:6 264:8||t="problem"']
set()


264
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'performed', 'and', 'whose', 'results', 'were', 'above', '1.5']
['c="radiograph" 265:4 265:4||t="test"']
['c="radiograph" 265:4 265:4||t="problem"']
set()


265
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'performed', 'and', 'whose', 'results', 'were', 'above', '4.9']
['c="radiograph" 266:4 266:4||t="test"']
['c="radiograph" 266:4 266:4||t="problem"']
set()


266
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'performed', 'and', 'whose', 'results', 'were', 'above', '3.54']
['c="radiograph" 267:4 267:4||t="test"']
['c="radiograph" 267:4 267:4||t="problem"', 'c="3.54" 267:12 267:12||t="problem"']
set()


267
['How', 'many', 'patients', 'were', 'tested', 'for', 'radi

[]
set()


308
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'between', '10', 'and', '49']
['c="radiograph" 309:4 309:4||t="test"']
['c="radiograph" 309:4 309:4||t="problem"']
set()


309
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'between', '30', 'and', '79']
['c="radiograph" 310:4 310:4||t="test"']
['c="radiograph" 310:4 310:4||t="problem"']
set()


310
['How', 'many', 'patients', 'had', 'labs', 'for', 'radiograph', 'whose', 'results', 'were', 'between', '10', 'and', '49']
['c="radiograph" 311:6 311:6||t="test"']
['c="labs" 311:4 311:4||t="test"']
set()


311
['How', 'many', 'patients', 'had', 'labs', 'for', 'radiograph', 'whose', 'results', 'were', 'between', '30', 'and', '79']
['c="radiograph" 312:6 312:6||t="test"']
['c="labs" 312:4 312:4||t="test"']
set()


312
['How', 'many', 'patients', 'had', 'results', 'for', 'radiograph', 'between', '10', 'and', '49']
['c="radiograph" 313:6 313:6||t="test"']
['c

['c="HIV values" 353:9 353:10||t="problem"']
set()


353
['How', 'many', 'people', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'values', 'greater', 'than', '4.9']
['c="immunodeficiency" 354:5 354:5||t="problem"', 'c="HIV" 354:9 354:9||t="test"']
['c="HIV values greater" 354:9 354:11||t="problem"']
set()


354
['How', 'many', 'people', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'values', 'greater', 'than', '3.54']
['c="immunodeficiency" 355:5 355:5||t="problem"', 'c="HIV" 355:9 355:9||t="test"']
['c="HIV values greater" 355:9 355:11||t="problem"']
set()


355
['How', 'many', 'patients', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'below', '1.5']
['c="immunodeficiency" 356:5 356:5||t="problem"', 'c="HIV" 356:9 356:9||t="test"']
[]
set()


356
['How', 'many', 'patients', 'suffering', 'from', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'below', '4.9']
['c="immunodeficiency" 357:5 357:5||t="probl

['c="administered Januvia lesser than" 402:4 402:7||t="problem"']
set()


402
['How', 'many', 'people', 'were', 'administered', 'Januvia', 'lesser', 'than', '3.54']
['c="Januvia" 403:5 403:5||t="treatment"']
['c="administered Januvia lesser than 3.54" 403:4 403:8||t="problem"']
set()


403
['How', 'many', 'people', 'were', 'administered', 'Digoxin', 'lesser', 'than', '1.5']
['c="Digoxin" 404:5 404:5||t="treatment"']
['c="administered Digoxin lesser" 404:4 404:6||t="problem"']
set()


404
['How', 'many', 'people', 'were', 'administered', 'Digoxin', 'lesser', 'than', '4.9']
['c="Digoxin" 405:5 405:5||t="treatment"']
['c="administered Digoxin lesser" 405:4 405:6||t="problem"']
set()


405
['How', 'many', 'people', 'were', 'administered', 'Digoxin', 'lesser', 'than', '3.54']
['c="Digoxin" 406:5 406:5||t="treatment"']
['c="administered Digoxin lesser than 3.54" 406:4 406:8||t="problem"']
set()


406
['How', 'many', 'people', 'were', 'administered', 'Januvia', 'dosage', 'below', '1.5']
['c="

In [53]:
import glob
train = glob.glob('train/*.con')
pred = glob.glob('pred_crf/*.con')
num_pred = 0
num_total = 0
num_inter = 0
for i in range(len(train)):
    #print(train[i], pred[i])
    n_t, n_p, n_i = util.evaluate(train[i], pred[i])
    num_pred += n_p
    num_total += n_t
    num_inter += n_i

precision:0.94
recall:0.58
f measure:0.72:
precision:1.00
recall:0.71
f measure:0.83:
precision:1.00
recall:0.61
f measure:0.76:
precision:0.98
recall:0.70
f measure:0.82:
precision:0.97
recall:0.55
f measure:0.70:
precision:0.97
recall:0.73
f measure:0.83:
precision:1.00
recall:0.70
f measure:0.82:
precision:1.00
recall:0.53
f measure:0.69:
precision:1.00
recall:0.54
f measure:0.70:
precision:1.00
recall:0.64
f measure:0.78:
precision:1.00
recall:0.73
f measure:0.85:
precision:1.00
recall:0.54
f measure:0.71:
precision:1.00
recall:0.61
f measure:0.75:
precision:1.00
recall:0.69
f measure:0.81:
precision:1.00
recall:0.54
f measure:0.70:
precision:1.00
recall:0.53
f measure:0.69:
precision:1.00
recall:0.70
f measure:0.82:
precision:1.00
recall:0.59
f measure:0.74:
precision:0.97
recall:0.68
f measure:0.80:
precision:0.93
recall:0.63
f measure:0.75:
precision:1.00
recall:0.60
f measure:0.75:
precision:0.98
recall:0.71
f measure:0.82:
precision:1.00
recall:0.68
f measure:0.81:
precision:1

In [54]:
precision = num_inter / num_pred
recall = num_inter / num_total
print("precision:{0:.2f}".format(precision))
print("recall:{0:.2f}".format(recall))
print("f measure:{0:.2f}".format(2 * precision * recall / (precision + recall)))

precision:0.98
recall:0.67
f measure:0.79


# Bilstm and crf with embedding

In [5]:
%run predict_crf_emb.py test_questions.txt test_questions_ground_truth.con lstm_crf_emb/epoch50.model

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class '__main__.BiLSTM_CRF' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the orig

0
['How', 'many', 'patients', 'suffer', 'from', 'Diabetes']
['c="Diabetes" 1:5 1:5||t="problem"']
[]
set()


1
['How', 'many', 'patients', 'suffer', 'from', 'heart', 'failure']
['c="heart failure" 2:5 2:6||t="problem"']
['c="heart failure" 2:5 2:6||t="problem"']
{'c="heart failure" 2:5 2:6||t="problem"'}


2
['How', 'many', 'patients', 'have', 'Diabetes']
['c="Diabetes" 3:4 3:4||t="problem"']
[]
set()


3
['How', 'many', 'patients', 'have', 'heart', 'failure']
['c="heart failure" 4:4 4:5||t="problem"']
['c="heart failure" 4:4 4:5||t="problem"']
{'c="heart failure" 4:4 4:5||t="problem"'}


4
['How', 'many', 'Diabetes', 'patients', 'have', 'been', 'admitted']
['c="Diabetes" 5:2 5:2||t="problem"']
['c="many Diabetes" 5:1 5:2||t="problem"']
set()


5
['How', 'many', 'heart', 'failure', 'patients', 'have', 'been', 'admitted']
['c="heart failure" 6:2 6:3||t="problem"']
['c="many heart failure" 6:1 6:3||t="problem"']
set()


6
['How', 'many', 'patients', 'have', 'been', 'administered', 'Januv

['c="heart failure" 48:5 48:6||t="problem"']
{'c="heart failure" 48:5 48:6||t="problem"'}


48
['Which', 'patients', 'expired', 'because', 'of', 'Diabetes']
['c="Diabetes" 49:5 49:5||t="problem"']
[]
set()


49
['Which', 'patients', 'expired', 'because', 'of', 'heart', 'failure']
['c="heart failure" 50:5 50:6||t="problem"']
['c="heart failure" 50:5 50:6||t="problem"']
{'c="heart failure" 50:5 50:6||t="problem"'}


50
['How', 'many', 'Diabetes', 'patients', 'died']
['c="Diabetes" 51:2 51:2||t="problem"']
['c="many Diabetes" 51:1 51:2||t="problem"']
set()


51
['How', 'many', 'heart', 'failure', 'patients', 'died']
['c="heart failure" 52:2 52:3||t="problem"']
['c="many heart failure" 52:1 52:3||t="problem"', 'c="died" 52:5 52:5||t="treatment"']
set()


52
['Which', 'Diabetes', 'patients', 'expired']
['c="Diabetes" 53:1 53:1||t="problem"']
['c="Diabetes" 53:1 53:1||t="problem"']
{'c="Diabetes" 53:1 53:1||t="problem"'}


53
['Which', 'heart', 'failure', 'patients', 'expired']
['c="heart fa

[]
set()


110
['How', 'many', 'BLACK', 'patients', 'have', 'immunodeficiency']
['c="immunodeficiency" 111:5 111:5||t="problem"']
[]
set()


111
['How', 'many', 'BLACK', 'patients', 'have', 'been', 'admitted', 'for', 'immunodeficiency']
['c="immunodeficiency" 112:8 112:8||t="problem"']
[]
set()


112
['How', 'many', 'patients', 'with', 'Diabetes', 'under', '1.5', 'age']
['c="Diabetes" 113:4 113:4||t="problem"']
['c="Diabetes" 113:4 113:4||t="problem"']
{'c="Diabetes" 113:4 113:4||t="problem"'}


113
['How', 'many', 'patients', 'with', 'Diabetes', 'under', '4.9', 'age']
['c="Diabetes" 114:4 114:4||t="problem"']
['c="Diabetes" 114:4 114:4||t="problem"']
{'c="Diabetes" 114:4 114:4||t="problem"'}


114
['How', 'many', 'patients', 'with', 'Diabetes', 'under', '3.54', 'age']
['c="Diabetes" 115:4 115:4||t="problem"']
['c="Diabetes" 115:4 115:4||t="problem"']
{'c="Diabetes" 115:4 115:4||t="problem"'}


115
['How', 'many', 'patients', 'with', 'heart', 'failure', 'under', '1.5', 'age']
['c="hear

['c="Diabetes" 169:5 169:5||t="problem"']
{'c="Diabetes" 169:5 169:5||t="problem"'}


169
['How', 'many', 'patients', 'suffering', 'from', 'heart', 'failure', 'were', 'under', '1.5', 'years']
['c="heart failure" 170:5 170:6||t="problem"']
['c="heart failure" 170:5 170:6||t="problem"']
{'c="heart failure" 170:5 170:6||t="problem"'}


170
['How', 'many', 'patients', 'suffering', 'from', 'heart', 'failure', 'were', 'under', '4.9', 'years']
['c="heart failure" 171:5 171:6||t="problem"']
['c="heart failure" 171:5 171:6||t="problem"']
{'c="heart failure" 171:5 171:6||t="problem"'}


171
['How', 'many', 'patients', 'suffering', 'from', 'heart', 'failure', 'were', 'under', '3.54', 'years']
['c="heart failure" 172:5 172:6||t="problem"']
['c="heart failure" 172:5 172:6||t="problem"']
{'c="heart failure" 172:5 172:6||t="problem"'}


172
['How', 'many', 'patients', 'suffering', 'from', 'Diabetes', 'were', 'between', '10', 'and', '49', 'years']
['c="Diabetes" 173:5 173:5||t="problem"']
['c="Diabete

['c="labs" 225:5 225:5||t="test"']
set()


225
['How', 'many', 'patients', 'had', 'positive', 'results', 'for', 'radiograph']
['c="radiograph" 226:7 226:7||t="test"']
['c="How many" 226:0 226:1||t="problem"', 'c="positive results" 226:4 226:5||t="problem"']
set()


226
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'performed', 'and', 'had', 'positive', 'results']
['c="radiograph" 227:4 227:4||t="test"']
['c="How many" 227:0 227:1||t="problem"', 'c="radiograph test" 227:4 227:5||t="test"']
set()


227
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'and', 'had', 'positive', 'results']
['c="radiograph" 228:6 228:6||t="test"']
['c="tested" 228:4 228:4||t="test"', 'c="radiograph" 228:6 228:6||t="treatment"']
set()


228
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'negative']
['c="radiograph" 229:6 229:6||t="test"']
['c="tested" 229:4 229:4||t="test"']
set()


229
['How', 'many', 'patients', 'had', 'negative', 'radiograph', 'test']
['c="r

['c="tested" 272:4 272:4||t="test"', 'c="radiograph whose results" 272:6 272:8||t="problem"']
set()


272
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'whose', 'results', 'were', 'lesser', 'than', '3.54']
['c="radiograph" 273:6 273:6||t="test"']
['c="radiograph" 273:6 273:6||t="test"']
{'c="radiograph" 273:6 273:6||t="test"'}


273
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'lesser', 'than', '1.5']
['c="radiograph" 274:4 274:4||t="test"']
['c="radiograph test" 274:4 274:5||t="test"']
set()


274
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'lesser', 'than', '4.9']
['c="radiograph" 275:4 275:4||t="test"']
['c="radiograph test" 275:4 275:5||t="test"']
set()


275
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'lesser', 'than', '3.54']
['c="radiograph" 276:4 276:4||t="test"']
['c="radiograph test" 276:4 276:5||t="test"']
set()


276
['How', 'many', 

['c="tested" 319:4 319:4||t="test"', 'c="radiograph whose results" 319:6 319:8||t="problem"']
set()


319
['How', 'many', 'patients', 'were', 'tested', 'for', 'radiograph', 'whose', 'results', 'were', 'between', '30', 'and', '79']
['c="radiograph" 320:6 320:6||t="test"']
['c="tested" 320:4 320:4||t="test"', 'c="radiograph" 320:6 320:6||t="test"']
{'c="radiograph" 320:6 320:6||t="test"'}


320
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'between', '10', 'and', '49']
['c="radiograph" 321:4 321:4||t="test"']
['c="radiograph test" 321:4 321:5||t="test"']
set()


321
['How', 'many', 'patients', 'had', 'radiograph', 'test', 'whose', 'results', 'were', 'between', '30', 'and', '79']
['c="radiograph" 322:4 322:4||t="test"']
['c="radiograph test" 322:4 322:5||t="test"']
set()


322
['How', 'many', 'patients', 'had', 'labs', 'for', 'radiograph', 'whose', 'results', 'were', 'between', '10', 'and', '49']
['c="radiograph" 323:6 323:6||t="test"']
['c="labs" 32

['c="many immunodeficiency" 362:1 362:2||t="problem"', 'c="HIV" 362:8 362:8||t="problem"']
set()


362
['How', 'many', 'immunodeficiency', 'patients', 'have', 'been', 'tested', 'for', 'HIV', 'below', '4.9']
['c="immunodeficiency" 363:2 363:2||t="problem"', 'c="HIV" 363:8 363:8||t="test"']
['c="HIV" 363:8 363:8||t="problem"']
set()


363
['How', 'many', 'immunodeficiency', 'patients', 'have', 'been', 'tested', 'for', 'HIV', 'below', '3.54']
['c="immunodeficiency" 364:2 364:2||t="problem"', 'c="HIV" 364:8 364:8||t="test"']
['c="many immunodeficiency" 364:1 364:2||t="problem"', 'c="HIV" 364:8 364:8||t="problem"']
set()


364
['How', 'many', 'people', 'with', 'immunodeficiency', 'were', 'tested', 'for', 'HIV', 'with', 'results', 'lesser', 'than', '1.5']
['c="immunodeficiency" 365:4 365:4||t="problem"', 'c="HIV" 365:8 365:8||t="test"']
['c="immunodeficiency" 365:4 365:4||t="test"', 'c="tested" 365:6 365:6||t="test"', 'c="HIV" 365:8 365:8||t="problem"']
set()


365
['How', 'many', 'people', 

['c="Digoxin dosage" 412:5 412:6||t="treatment"']
set()


412
['How', 'many', 'people', 'were', 'given', 'Januvia', 'dosage', 'below', '1.5', 'in', 'the', 'hospital']
['c="Januvia" 413:5 413:5||t="treatment"']
['c="Januvia dosage" 413:5 413:6||t="treatment"']
set()


413
['How', 'many', 'people', 'were', 'given', 'Januvia', 'dosage', 'below', '4.9', 'in', 'the', 'hospital']
['c="Januvia" 414:5 414:5||t="treatment"']
['c="Januvia dosage" 414:5 414:6||t="treatment"']
set()


414
['How', 'many', 'people', 'were', 'given', 'Januvia', 'dosage', 'below', '3.54', 'in', 'the', 'hospital']
['c="Januvia" 415:5 415:5||t="treatment"']
['c="Januvia dosage" 415:5 415:6||t="treatment"']
set()


415
['How', 'many', 'people', 'were', 'given', 'Digoxin', 'dosage', 'below', '1.5', 'in', 'the', 'hospital']
['c="Digoxin" 416:5 416:5||t="treatment"']
['c="Digoxin dosage" 416:5 416:6||t="treatment"']
set()


416
['How', 'many', 'people', 'were', 'given', 'Digoxin', 'dosage', 'below', '4.9', 'in', 'the', 'hos

In [94]:
%run predict_crf_emb.py train/1.txt train/1.con lstm_crf_emb/epoch50.model

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class '__main__.BiLSTM_CRF' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the orig

0
['Admission', 'Date', ':']
['c="Admission" 1:0 1:0||t="occurrence"']
[]
set()


1
['09/29/1993']
[]
[]
set()


2
['Discharge', 'Date', ':']
['c="Discharge" 3:0 3:0||t="occurrence"']
[]
set()


3
['10/04/1993']
[]
[]
set()


4
['HISTORY', 'OF', 'PRESENT', 'ILLNESS', ':']
[]
[]
set()


5
['The', 'patient', 'is', 'a', '28-year-old', 'woman', 'who', 'is', 'HIV', 'positive', 'for', 'two', 'years', '.']
['c="HIV positive" 6:8 6:9||t="problem"']
['c="HIV positive" 6:8 6:9||t="problem"']
{'c="HIV positive" 6:8 6:9||t="problem"'}


6
['She', 'presented', 'with', 'left', 'upper', 'quadrant', 'pain', 'as', 'well', 'as', 'nausea', 'and', 'vomiting', 'which', 'is', 'a', 'long-standing', 'complaint', '.']
['c="presented" 7:1 7:1||t="evidential"', 'c="left upper quadrant pain" 7:3 7:6||t="problem"', 'c="nausea" 7:10 7:10||t="problem"', 'c="vomiting" 7:12 7:12||t="problem"', 'c="a long-standing complaint" 7:15 7:17||t="problem"']
['c="left upper quadrant pain" 7:3 7:6||t="problem"', 'c="nausea" 7:10

['c="some pain" 27:9 27:10||t="problem"', 'c="Percocet" 27:15 27:15||t="treatment"']
{'c="some pain" 27:9 27:10||t="problem"', 'c="Percocet" 27:15 27:15||t="treatment"'}


27
['She', 'went', 'for', 'a', 'debridement', 'of', 'her', 'left', 'calf', 'lesion', 'on', '10/2/93', 'and', 'was', 'started', 'empirically', 'on', 'IV', 'ceftriaxone', 'which', 'was', 'changed', 'to', 'po', 'doxycycline', 'on', 'the', 'day', 'of', 'discharge', '.']
['c="a debridement" 28:3 28:4||t="treatment"', 'c="her left calf lesion" 28:6 28:9||t="problem"', 'c="IV ceftriaxone" 28:17 28:18||t="treatment"', 'c="po doxycycline" 28:23 28:24||t="treatment"', 'c="discharge" 28:29 28:29||t="occurrence"']
['c="a debridement" 28:3 28:4||t="treatment"', 'c="her left calf lesion" 28:6 28:9||t="problem"', 'c="IV ceftriaxone" 28:17 28:18||t="treatment"', 'c="po doxycycline" 28:23 28:24||t="treatment"']
{'c="IV ceftriaxone" 28:17 28:18||t="treatment"', 'c="her left calf lesion" 28:6 28:9||t="problem"', 'c="a debridement" 28:3

In [3]:
import glob
train = glob.glob('train/*.con')
pred = glob.glob('pred_crf_emb/*.con')
num_pred = 0
num_total = 0
num_inter = 0
for i in range(len(train)):
    #print(train[i], pred[i])
    n_t, n_p, n_i = util.evaluate(train[i], pred[i])
    num_pred += n_p
    num_total += n_t
    num_inter += n_i

precision:1.00
recall:0.62
f measure:0.76:
precision:1.00
recall:0.71
f measure:0.83:
precision:1.00
recall:0.61
f measure:0.76:
precision:0.99
recall:0.71
f measure:0.83:
precision:1.00
recall:0.55
f measure:0.71:
precision:0.99
recall:0.74
f measure:0.84:
precision:1.00
recall:0.70
f measure:0.82:
precision:0.96
recall:0.53
f measure:0.68:
precision:1.00
recall:0.54
f measure:0.70:
precision:0.97
recall:0.64
f measure:0.77:
precision:1.00
recall:0.73
f measure:0.85:
precision:1.00
recall:0.54
f measure:0.71:
precision:1.00
recall:0.61
f measure:0.75:
precision:0.97
recall:0.67
f measure:0.79:
precision:1.00
recall:0.54
f measure:0.70:
precision:1.00
recall:0.54
f measure:0.70:
precision:1.00
recall:0.70
f measure:0.82:
precision:1.00
recall:0.60
f measure:0.75:
precision:1.00
recall:0.70
f measure:0.82:
precision:0.98
recall:0.66
f measure:0.79:
precision:1.00
recall:0.60
f measure:0.75:
precision:1.00
recall:0.72
f measure:0.84:
precision:0.98
recall:0.68
f measure:0.80:
precision:1

In [4]:
precision = num_inter / num_pred
recall = num_inter / num_total
print("precision:{0:.2f}".format(precision))
print("recall:{0:.2f}".format(recall))
print("f measure:{0:.2f}".format(2 * precision * recall / (precision + recall)))

precision:0.99
recall:0.67
f measure:0.80
